# Assignment 01: Multi-class Classification 
In this Assignment, you will train a deep model on the CIFAR10 from the scratch using PyTorch.

### Basic Imports

In [1]:
import os
import time
import os.path as osp

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms
import torchvision

import matplotlib.pyplot as plt
from PIL import Image

### Hyperparameters

In [2]:
# random seed
SEED = 1 
NUM_CLASS = 10

# Training
BATCH_SIZE = 128
NUM_EPOCHS = 3
EVAL_INTERVAL=1
SAVE_DIR = './log'

# Optimizer
LEARNING_RATE = 1e-1
MOMENTUM = 0.9
STEP=5
GAMMA=0.5


### Device

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


### Dataset


In [4]:
# cifar10 transform
transform_cifar10_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_cifar10_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_set = torchvision.datasets.CIFAR10(root='../data', train=True,
                                        download=True, transform=transform_cifar10_train)
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(root='../data', train=False,
                                       download=True, transform=transform_cifar10_test)
test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


Files already downloaded and verified
Files already downloaded and verified


### Model

In [5]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 4, 3)  
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(4, 8, 3)  
        self.fc1 = nn.Linear(8 * 6 * 6, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 8 * 6 * 6)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
model = ConvNet()
model.to(device)

ConvNet(
  (conv1): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=288, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=10, bias=True)
)

### Optimizer

In [7]:
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP, gamma=GAMMA)

### Task 1: per batch training/testing
---

Please denfine two function named ``train_batch`` and ``test_batch``. These functions are essential for training and evaluating machine learning models using batched data from dataloaders.

**To do**: 
1. Define the loss function i.e [nn.CrossEntropyLoss()](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html).
2. Take the image as the input and generate the output using the pre-defined SimpleNet.
3. Calculate the loss between the output and the corresponding label using the loss function.

In [8]:
def focal_loss(pred, target, alpha=0.25, gamma=5):
    ce_loss = torch.nn.functional.cross_entropy(pred, target, reduction='none')
    pt = torch.exp(-ce_loss)
    focal_loss = alpha * (1 - pt) ** gamma * ce_loss
    return focal_loss.mean()


In [9]:
##################### Write your answer here ##################
# Define the loss function
#criterion = torch.nn.L1Loss()
#criterion = torch.nn.CrossEntropyLoss()
criterion = nn.MSELoss()
###############################################################

In [16]:
def train_batch(model, image, target):
    """
    Args:
        model (torch.nn.Module): The machine learning model to train.
        image (torch.Tensor): Batch of input data (images).
        target (torch.Tensor): Batch of target labels.
    Returns:
        torch.Tensor: Model output (predictions) for the batch.
        torch.Tensor: Loss value calculated by the defined loss function loss_fn().
    """ 
    output = model(image)
    one_hot_target = nn.functional.one_hot(target,10)
    output = F.softmax(output, dim=1)
    #loss = criterion(output,target)
    loss = criterion(output,one_hot_target)
    #loss = criterion(output,target.float)
    #loss = focal_loss(output,target)
    return output, loss

In [17]:

def test_batch(model, image, target):
    """
    Args:
        model (torch.nn.Module): The machine learning model to evaluate.
        image (torch.Tensor): Batch of input data (images).
        target (torch.Tensor): Batch of target labels.
    Returns:
        torch.Tensor: Model output (predictions) for the batch.
        torch.Tensor: Loss value calculated for the batch.
    """
    output = model(image)
    one_hot_target = nn.functional.one_hot(target,10)
    output = F.softmax(output, dim=1)
    loss = criterion(output,one_hot_target)
    #loss = criterion(output,target.float)
    #loss = focal_loss(output,target)
    return output, loss

### Model Training

In [18]:
training_loss = []
training_acc = []
testing_loss = []
testing_acc = []

with open('MSE_Loss_result.txt', 'w') as file:

    for epoch in range(NUM_EPOCHS):

        model.train()
        torch.cuda.empty_cache()

    ##########################
    ### Training
    ##########################

        running_cls_loss = 0.0
        running_cls_corrects = 0

        for batch_idx, (image, target) in enumerate(train_dataloader):

            image = image.to(device)
            target = target.to(device)

        # train model
            outputs, loss = train_batch(model, image, target)
            _, preds = torch.max(outputs, 1)


            loss_data = loss.data.item()
            if np.isnan(loss_data):
                raise ValueError('loss is nan while training')
            running_cls_loss += loss.item()
            running_cls_corrects += torch.sum(preds == target.data)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        epoch_loss = running_cls_loss / len(train_set)
        epoch_acc = running_cls_corrects.double() / len(train_set)

        print(f'Epoch: {epoch+1}/{NUM_EPOCHS} Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        file.write(f'Epoch: {epoch+1}/{NUM_EPOCHS} Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}\n')

        training_loss.append(epoch_loss)
        training_acc.append(epoch_acc.cpu().detach().numpy())

    # change learning rate
        scheduler.step()


    ##########################
    ### Testing
    ##########################
    # # eval model during training or in the last epoch
        if (epoch + 1) % EVAL_INTERVAL == 0 or (epoch +1) == NUM_EPOCHS:
            print('Begin test......')
            file.write('Begin test......\n')
            model.eval()

            val_loss = 0.0
            val_corrects = 0

            for batch_idx, (image, target) in enumerate(test_dataloader):

                image = image.to(device)
                target = target.to(device)

            # test model
                outputs, loss = test_batch(model, image, target)
                _, preds = torch.max(outputs, 1)

                val_loss += loss.item()
                val_corrects += torch.sum(preds == target.data)

            val_loss = val_loss / len(test_set)
            val_acc = val_corrects.double() / len(test_set)
            print(f'Test Loss: {val_loss:.4f} Acc: {val_acc:.4f}')
            file.write(f'Test Loss: {val_loss:.4f} Acc: {val_acc:.4f}')
            testing_loss.append(val_loss)
            testing_acc.append(val_acc.cpu().detach().numpy())

        # save the model in last epoch
            if (epoch +1) == NUM_EPOCHS:

                state = {
                'state_dict': model.state_dict(),
                'acc': epoch_acc,
                'epoch': (epoch+1),
                }

            # check the dir
                if not os.path.exists(SAVE_DIR):
                    os.makedirs(SAVE_DIR)

            # save the state
                torch.save(state, osp.join(SAVE_DIR, 'checkpoint_%s.pth' % (str(epoch+1))))

AttributeError: 'builtin_function_or_method' object has no attribute 'size'

In [ ]:
def Vis(data,y):
    plt.plot(data)
    plt.xlabel('epoch')
    plt.ylabel(y)
    plt.show()
    
Vis(training_loss,'training_loss')
Vis(training_acc,'training_acc')
Vis(testing_loss,'testing_loss')
Vis(testing_acc,'testing_acc')

def WriteData(training_loss,training_acc,testing_loss,testing_acc,fuc):
    with open(fuc+'_data_TrainLoss.txt', 'w') as file:
        for item in training_loss:
            file.write(str(item)+'\n')
            
    with open(fuc+'_data_TrainAcc.txt', 'w') as file:
        value_list = [tensor.item() for tensor in training_acc]
        for item in value_list:
            file.write(str(item)+'\n')
            
    with open(fuc+'_data_TestLoss.txt', 'w') as file:
        for item in testing_loss:
            file.write(str(item)+'\n')
            
    with open(fuc+'_data_TestAcc.txt', 'w') as file:
        value_list = [tensor.item() for tensor in testing_acc]
        for item in value_list:
            file.write(str(item)+'\n')
        
      
def ReadData(file):
    with open(file, 'r') as file:
        data_list=[float(line) for line in file]
        return data_list
    
#WriteData(training_loss,training_acc,testing_loss,testing_acc,'Fo5')



In [ ]:
def plotdata(file):
    plt.figure()
    plt.plot(ReadData('CE_data_'+file+'.txt'), label='CE', marker='o', linestyle='-')
    plt.plot(ReadData('L1_data_'+file+'.txt'), label='L1', marker='s', linestyle='--')
    plt.plot(ReadData('Fo0.5_data_'+file+'.txt'), label='Fo2', marker='x', linestyle='-.')
    plt.plot(ReadData('Fo2_data_'+file+'.txt'), label='Fo0.5', marker='^', linestyle=':')
    plt.plot(ReadData('Fo5_data_'+file+'.txt'), label='Fo5', marker='d', linestyle='-')
    plt.legend()
    plt.title(file)
    plt.show()
plotdata('TrainAcc')
plotdata('TrainLoss')
plotdata('TestAcc')
plotdata('TestLoss')

### Task 2: Instance inference
---
The task is to visualizes an image along with model prediction and class probabilities.

**To do**: 
1. Calculate the prediction and the probabilities for each class.
         

In [ ]:
inputs, classes = next(iter(test_dataloader))
input = inputs[0]

In [ ]:
##################### Write your answer here ##################
# input: image, model
# outputs: predict_label, probabilities
# predict_label is the index (or label) of the class with the highest probability from the probabilities.
###############################################################

probabilities = F.softmax(model(input.to(device)),dim=1)          #得到一个概率向量
predict_label = torch.argmax(probabilities)           #得到一个概率，最大概率

In [ ]:
predicted_class = class_names[predict_label.item()]
predicted_probability = probabilities[0][predict_label].item()
image = input.numpy().transpose((1, 2, 0))
plt.imshow(image)
plt.text(17, 30, f'Predicted Class: {predicted_class}\nProbability: {predicted_probability:.2f}', 
            color='white', backgroundcolor='black', fontsize=8)
plt.show()

# Print probabilities for each class
print('Print probabilities for each class:')
for i in range(len(class_names)):
    print(f'{class_names[i]}: {probabilities[0][i].item():.4f}')